# IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pickle
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import torch
import torch.nn as nn
from transformers import BertTokenizerFast

# FLASK

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', do_lower_case=True)

In [21]:
class RNN(nn.Module):
    def __init__(self, hidden_size, embedding_size, num_layers=2, dropout=0.3):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.embedding = nn.Embedding(tokenizer.vocab_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, num_layers = num_layers, batch_first=True)
        self.gru = nn.GRU(embedding_size, hidden_size, num_layers = num_layers, batch_first=True)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers = num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size*30,hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size//2)
        self.fc3 = nn.Linear(hidden_size//2,3)
        self.relu =  nn.ReLU()
    def forward(self, input_sequence, model_type):
        embedded = self.embedding(input_sequence)
        if model_type == "RNN":
            outputs, hidden = self.rnn(embedded)
        elif model_type == "GRU":
            outputs, hidden = self.gru(embedded)
        elif model_type == "LSTM":
            outputs, (hidden, cell) = self.lstm(embedded)
        x = self.fc1(outputs.contiguous().view(-1, 30 * self.hidden_size))
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [22]:
model = RNN(128, 200, 1, 0.3)

In [23]:
model.load_state_dict(torch.load('saved_lstm_weights.pt'))

<All keys matched successfully>

In [43]:
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    encoded_sent = tokenizer.encode(request.form['text'], add_special_tokens = False, max_length = 30, truncation = True, pad_to_max_length='right')    
    features = torch.tensor([encoded_sent])
    
    # PREDICTION FROM SAVED MODEL
    model.eval()
    with torch.no_grad():
        preds = model(features, "LSTM")
        result = np.argmax(nn.LogSoftmax(dim=1)(preds).detach().cpu().numpy(), axis=1)[0]
    if result == 0:
        result = "<h3> <p>You are &#128548;</p> <p>We are sorry for the inconvenience</p></h3>"
    elif result == 1:
        result = "<h3> <p>You are &#128512;</p> <p>Thankyou! Atleast we are not bad</p></h3>"
    else:
        result = "<h3> <p>You are &#128513;</p> <p>Thankyou! hope you fly with us again</p></h3>"
    
    # SHOWING RESULT TO USER
    return render_template(
        "index.html",
        prediction_text="{}".format(result),
    )

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Sep/2021 00:24:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2021 00:24:40] "POST /predict HTTP/1.1" 200 -


 * Running on http://f3d1-103-226-144-197.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [16/Sep/2021 00:24:45] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2021 00:24:49] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2021 00:24:57] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2021 00:25:01] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2021 00:25:41] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/Sep/2021 00:25:48] "POST /predict HTTP/1.1" 200 -
